In [74]:
# https://youtu.be/QogFIWa1YMg?list=PLA0M1Bcd0w8ynD1umfubKq1OBYRXhXkmH
%env TF_CPP_MIN_LOG_LEVEL=2

import tensorflow as tf  # noqa: E402
from tensorflow.keras.datasets import mnist  # noqa: E402
from tensorflow.keras.utils import to_categorical  # noqa: E402


class DenseLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units  # number of neurons

    def build(self, input_shape):
        self.w = self.add_weight(
            name="w",
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            name="b",
            shape=(self.units,),
            initializer="zeros",
            trainable=True,
        )

    def call(self, inputs):
        # regular = 100.0 * tf.reduce_mean(tf.square(self.w))
        # self.add_loss(regular)
        return tf.matmul(inputs, self.w) + self.b


env: TF_CPP_MIN_LOG_LEVEL=2


In [75]:
class NeuralNetwork(tf.keras.Model):
    def __init__(self, num_classes=10):
        super().__init__()
        self.layer_1 = DenseLayer(128)
        self.layer_2 = DenseLayer(num_classes)

    def call(self, inputs):
        x = self.layer_1(inputs)
        x = tf.nn.relu(x)
        x = self.layer_2(x)
        x = tf.nn.softmax(x)
        return x

In [76]:
model = NeuralNetwork()

# model.compile(
#     # optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#     optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),  # for M1/M2 Mac
#     loss=tf.losses.CategoricalCrossentropy(),
#     metrics=["accuracy"],
# )

model.compile(
    optimizer="adam",
    # optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001),  # for M1/M2 Mac
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [70]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess data
x_train = tf.cast(x_train.reshape((-1, 28 * 28)) / 255.0, tf.float32)
x_test = tf.cast(x_test.reshape((-1, 28 * 28)) / 255.0, tf.float32)

y_train = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

In [77]:
# Learning...
model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=10,
    # validation_data=(x_test, y_test_cat),
    # verbose=2,
)

Epoch 1/10
1875/1875 [==============================] - 1s 604us/step - loss: 0.2855 - accuracy: 0.9205
Epoch 2/10
1875/1875 [==============================] - 1s 534us/step - loss: 0.1206 - accuracy: 0.9643
Epoch 3/10
1875/1875 [==============================] - 1s 625us/step - loss: 0.0824 - accuracy: 0.9754
Epoch 4/10
1875/1875 [==============================] - 1s 544us/step - loss: 0.0616 - accuracy: 0.9809
Epoch 5/10
1875/1875 [==============================] - 1s 551us/step - loss: 0.0475 - accuracy: 0.9849
Epoch 6/10
1875/1875 [==============================] - 1s 560us/step - loss: 0.0378 - accuracy: 0.9883
Epoch 7/10
1875/1875 [==============================] - 1s 544us/step - loss: 0.0307 - accuracy: 0.9906
Epoch 8/10
1875/1875 [==============================] - 1s 594us/step - loss: 0.0252 - accuracy: 0.9924
Epoch 9/10
1875/1875 [==============================] - 1s 579us/step - loss: 0.0192 - accuracy: 0.9944
Epoch 10/10
1875/1875 [==============================] - 1s 591u

In [78]:
# Testing...
model.evaluate(x_test, y_test_cat)

313/313 [==============================] - 0s 365us/step - loss: 0.0789 - accuracy: 0.9765


[0.07887882739305496, 0.9764999747276306]

In [73]:
tf.random.uniform((1, 10), minval=0, maxval=1, dtype=tf.int32)

<tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>